In [ ]:
import pymongo
from pymongo import MongoClient
import os

# Example minimal queries

In [ ]:
db_name: str = "myntra"
collection_name: str = "myntra-items-offline"

In [ ]:
client = MongoClient(os.environ["DB_CONNECTION_STRING"])
collection = client[db_name][collection_name]

## Combining text query and metadata filtering

In [ ]:
text_search = 'dress'
text_search_index_name = 'lexical_text_search_index'
min_price=0
max_price=10_000
min_rating=0
n=10

In [ ]:
def lexical_search(text_search: str, text_search_index_name: str) -> list[dict]:
    return [
        {
            "$search": {
                "index": text_search_index_name,
                "text": {
                    "query": text_search,
                    "path": "name",
                },
            }
        }
    ]


In [ ]:
def match_on_metadata(
    min_price: int,
    max_price: int,
    min_rating: float,
    n: int,
    categories: list[str] | None = None,
    colors: list[str] | None = None,
    seasons: list[str] | None = None,
) -> list[dict]:
    match_spec = {
        "price": {
            "$gte": min_price,
            "$lte": max_price,
        },
        "rating": {"$gte": min_rating},
    }
    if categories:
        match_spec["category"] = {"$in": categories}
    if colors:
        match_spec["color"] = {"$in": colors}
    if seasons:
        match_spec["season"] = {"$in": seasons}

    return [
        {
            "$match": match_spec,
        },
        {"$limit": n},
    ]


In [ ]:
pipeline = []
if text_search.strip():
    pipeline.extend(
        lexical_search(
            text_search=text_search,
            text_search_index_name=text_search_index_name,
        )
    )

pipeline.extend(
    match_on_metadata(
        min_price=min_price,
        max_price=max_price,
        min_rating=min_rating,
        n=n,
    )
)

records = collection.aggregate(pipeline)
results = [
    (record["img"].split(";")[-1].strip(), record["name"])
    # async 
    for record in records
]

n_results = len(results)

In [ ]:
results

In [ ]:
n_results

## Example vector search

Embedding for the text "dress"

In [ ]:
query_vector = [-0.024136465042829514, -0.023728929460048676, -0.03186793625354767, -0.01082341093569994, -0.0150108328089118, 0.016808178275823593, 0.013221118599176407, 0.020104017108678818, 0.04778704419732094, 0.036966968327760696, 0.061754439026117325, -0.00010883400682359934, 0.0340559147298336, -0.03324975073337555, -0.007400914561003447, 0.03463617339730263, -0.036423392593860626, -0.062387656420469284, -0.020940301939845085, 0.030518634244799614, -0.03109283745288849, 0.008197261020541191, -0.10206734389066696, -0.031112831085920334, -0.03018222376704216, 0.05737512186169624, 0.02595696412026882, -0.01188658643513918, 0.034131456166505814, 0.04093340411782265, -0.03607457876205444, -0.0022378929425030947, 0.013730671256780624, -0.04829508811235428, -0.018866294994950294, -0.009474823251366615, 0.023982089012861252, -0.0374121256172657, -0.02453644946217537, -0.047164227813482285, 0.0205993615090847, -0.018402118235826492, 0.03555254265666008, -0.03199446573853493, -0.04303589090704918, -0.013559038750827312, 0.009827944450080395, -0.06439293920993805, 0.01570974476635456, -0.007694865111261606, 0.014262297190725803, -0.02435862272977829, -0.02107273042201996, -0.0026534656062722206, -0.0022659788373857737, 0.004232665523886681, 0.01789276860654354, -0.019983308389782906, -0.033662762492895126, -0.019748598337173462, 0.02828417532145977, -0.006983633618801832, 0.01270337961614132, -0.0661192536354065, 0.03286021202802658, 0.03306064382195473, -0.0036307754926383495, 0.009994429536163807, -0.009047215804457664, -0.01783219538629055, 0.02865288034081459, 0.01264595240354538, -0.0048859575763344765, -0.01729229837656021, -0.0047726440243422985, 0.0006044947658665478, -0.023932592943310738, -0.0010761216981336474, -0.0008062702254392207, 0.011442938819527626, 0.04319433122873306, 0.03948844596743584, 0.002560243709012866, 0.016901226714253426, -0.04001779854297638, -0.04067693650722504, -0.018264926970005035, 0.030025199055671692, 0.002787400037050247, 0.008525812067091465, -0.01818443462252617, 0.05514531955122948, -0.0035623395815491676, -0.015101402066648006, 0.02167498506605625, 0.049612291157245636, -0.03224371373653412, 0.018000302836298943, 0.016536416485905647, 0.019323576241731644, 0.032353684306144714, 0.026254400610923767, -0.0017869853181764483, 0.04761822149157524, -0.04318494349718094, 0.014297629706561565, 0.01058721262961626, -0.025402626022696495, -0.03606828302145004, -0.038587845861911774, 0.016939446330070496, -0.031492382287979126, 0.03160407394170761, 0.0041166129522025585, -0.011724643409252167, 0.04952608421444893, 0.012676567770540714, 0.027847399935126305, -0.020507030189037323, 0.002019185572862625, 0.028075164183974266, 0.021089721471071243, 0.009731605648994446, -0.020754646509885788, 0.019544892013072968, -0.04115282744169235, -0.033457450568675995, 0.0892089381814003, -0.023721512407064438, -0.004559481516480446, 0.008517077192664146, -0.03142441436648369, 0.01797575317323208, 0.033843737095594406, -0.02061385288834572, 0.020094718784093857, -0.031926028430461884, 0.017796428874135017, 0.001020433846861124, -0.07218600064516068, 0.06783035397529602, 0.023565558716654778, -0.018844427540898323, 0.08426930010318756, 0.03665178269147873, 0.04787234961986542, 0.030529847368597984, 0.002824636874720454, -0.038247279822826385, -0.00033479975536465645, -0.015567732974886894, 0.026125576347112656, -0.014239468611776829, 0.03622389957308769, 0.029984503984451294, 0.006184131372720003, -0.0034912247210741043, -0.012457478791475296, 0.002699977718293667, 0.023633498698472977, -0.009476094506680965, 0.004161766730248928, -0.022885851562023163, 0.02100110799074173, 0.002005532383918762, 0.02960014156997204, -0.03621998429298401, -0.007284307852387428, -0.006691812537610531, -0.02920294553041458, 0.009404204785823822, -0.000725898309610784, 0.007698355242609978, 0.0011397459311410785, -0.0035187334287911654, 0.05228735879063606, 0.03257794678211212, -0.0041107297874987125, 0.018327724188566208, 0.02347324602305889, -0.017657630145549774, -0.006876824423670769, -0.037516556680202484, 0.056226834654808044, 0.010437414981424809, -0.010648004710674286, 0.013965846039354801, -0.017816314473748207, -0.04828454181551933, -0.039009157568216324, 0.012892982922494411, 0.021801088005304337, -0.020459666848182678, 0.0238940492272377, 0.04361308366060257, 0.021244050934910774, -0.01140366867184639, 0.021489137783646584, -0.02046596072614193, -0.03192463889718056, -0.02486291714012623, 0.05358368903398514, -0.05018409341573715, 0.011774512939155102, 0.027243604883551598, -0.030060892924666405, 0.035313479602336884, 0.031342506408691406, -0.05820516496896744, 0.014512723311781883, 0.04629703238606453, -0.0004279354470781982, 0.0058724540285766125, -0.008254481479525566, 0.01707361824810505, -0.01144208014011383, -0.034239765256643295, 0.039355892688035965, -0.02448633499443531, -0.0025807558558881283, 0.010931854136288166, 0.0355960913002491, 0.044449009001255035, 0.03646925091743469, -0.026959165930747986, -0.009241391904652119, 0.017054572701454163, 0.06519397348165512, -0.004746947903186083, 0.02028416097164154, -0.010715944692492485, 0.0566108301281929, 0.03146633133292198, 0.03768735006451607, 0.04316890239715576, 0.009603815153241158, 0.031991127878427505, 0.030562693253159523, -0.03921894729137421, 0.051937852054834366, 0.00465276138857007, 0.010889410972595215, 0.02541661448776722, 0.035771600902080536, -0.013572142459452152, 0.040570277720689774, 0.0021343843545764685, -0.009320247918367386, -0.013280742801725864, 0.028611017391085625, 0.023610794916749, 0.042209647595882416, 0.034439101815223694, 0.04351917654275894, -0.02910654991865158, 0.017301995307207108, 0.029629642143845558, 0.07915602624416351, -0.011150387115776539, 0.0030616705771535635, 0.00605747802183032, 0.018452560529112816, -0.01935412734746933, 0.028892822563648224, 0.013913779519498348, 0.02795044332742691, 0.009615649469196796, 0.02493513934314251, -0.0039544543251395226, -0.0473797507584095, -0.047195952385663986, -0.04437507316470146, -0.025916771963238716, -0.029634589329361916, -0.01014145277440548, -0.001198369194753468, 0.02128221094608307, -0.03416239097714424, 0.04883786663413048, 0.0220356322824955, -0.03184516727924347, -0.061647336930036545, -0.02337268553674221, 0.07285881787538528, 0.005595417693257332, 0.026931721717119217, -0.03478135168552399, 0.032939255237579346, -0.004723012447357178, 0.04429890960454941, -0.052908267825841904, -0.02107255533337593, 0.012674191035330296, -0.007246042136102915, 0.05158349871635437, -0.03492041677236557, -0.020263422280550003, 0.00308004068210721, -0.045279331505298615, -0.07180838286876678, -0.00793076679110527, 0.020312152802944183, -0.03555338829755783, 0.017932122573256493, -0.049126580357551575, 0.04453456774353981, 0.0251720380038023, -0.025084728375077248, 0.04691745713353157, 0.02653566189110279, -0.05629954859614372, 0.018988817930221558, 0.02960270084440708, -0.007907168008387089, -0.05775388702750206, 0.05932541936635971, 0.05756717547774315, -0.0017637134296819568, -0.025688422843813896, -0.0035485397092998028, -0.04071512073278427, -0.0222929734736681, 0.020771561190485954, 0.003261130303144455, -0.03628822788596153, 0.029039397835731506, 0.014385415241122246, -0.08968624472618103, 0.020115850493311882, -0.029533090069890022, -0.03988682106137276, -0.05137473717331886, -0.035710833966732025, 0.023409154266119003, 0.03776099532842636, -6.472045060945675e-05, -0.02103831246495247, -0.010178589262068272, 0.003695087041705847, 0.021520432084798813, 0.03831468150019646, -0.02052910439670086, 0.04889426380395889, 0.0462011955678463, -0.013410990126430988, 0.005569192580878735, 0.012486474588513374, -0.023895632475614548, -0.003931901417672634, 0.027597758919000626, -0.001578913303092122, 0.017619002610445023, 0.029947640374302864, 0.0074917022138834, -0.011757146567106247, 0.019682101905345917, -0.042260706424713135, -0.030111977830529213, -0.004962125327438116, -0.007144961040467024, 0.030902069061994553, 0.0147103788331151, 0.00890989601612091, -0.03522372990846634, -0.021479228511452675, -0.05341045558452606, 0.03047531098127365, -0.002037174766883254, 0.031798649579286575, -0.05343581363558769, 0.04313056543469429, -0.0017008042195811868, -0.046101197600364685, 0.05441506952047348, -0.06114301085472107, 0.00013871588453184813, 0.05054549127817154, 0.005809216760098934, 0.024418434128165245, -0.027752192690968513, 0.0051126196049153805, -0.0391683392226696, 0.009865165688097477, 0.03148941695690155, 0.0033974298276007175, 0.009123885072767735, -0.04306021332740784, -0.01097997184842825, -0.029876379296183586, -0.027562560513615608, 0.0022206061985343695, 0.004676004871726036, 0.017239216715097427, -0.017488526180386543, -0.0402672216296196, -0.03384297713637352, 0.028313562273979187, 0.0295184887945652, 0.02074993960559368, 0.03317086398601532, 0.046707283705472946, 0.011596030555665493, 0.014065463095903397, 0.041696708649396896, -0.05310836434364319, 0.04447789490222931, -0.056267354637384415, 0.04015779867768288, 0.022455278784036636, -0.0025771090295165777, -0.011421714909374714, -0.02249634638428688, -0.01714319735765457, 0.024908626452088356, 0.02568625845015049, -0.032582078129053116, -0.04875703528523445, 0.019736746326088905, 0.005784394685178995, 0.03638589009642601, -0.024627715349197388, -0.04112233966588974, 0.016171235591173172, 0.02308547869324684, 0.02366534247994423, -0.030561599880456924, 0.02236560545861721, -0.02389754354953766, 0.039137158542871475, 0.038774315267801285, -0.013904178515076637, -0.02557908371090889, -0.021563535556197166, -0.042091358453035355, -0.04108969867229462, -0.024592064321041107, 0.036652132868766785, 0.0032373329158872366, 0.03800315037369728, -0.0259995236992836, 0.02721782587468624, 0.03596947342157364, 0.0034172344021499157, 0.0027664408553391695, 0.0073640719056129456, 0.011904116719961166, 0.008274887688457966, 0.04355943948030472, -0.018800662830471992, -0.0060120546258986, 0.023152261972427368, -0.06641967594623566, 0.030216369777917862, 0.0011208064388483763, -0.0012782829580828547, 0.0010003423085436225, 0.007205942180007696, 0.034843359142541885, 0.021076340228319168, -0.027115458622574806, 0.022761935368180275, -0.044145576655864716, 0.021388037130236626, -0.0567937009036541, -0.045064687728881836, 0.08522528409957886, -0.02075996808707714, -0.020261425524950027, 0.04359766095876694, 0.04410471022129059, -0.0035222102887928486, -0.009108681231737137, 0.02646227739751339, -0.03554845228791237, 0.01779068075120449, -0.03581691160798073, -0.010217325761914253, -0.027149328961968422, -0.04389818757772446, -0.007348119746893644, -0.02076849900186062, 0.013682805001735687, 0.009809782728552818, -0.03494220972061157, -0.03405246511101723, -0.0776825100183487, -0.030826415866613388, 0.007848965935409069, -0.009186530485749245, 0.024486856535077095, -0.015498635359108448, -0.024391451850533485, -0.015553277917206287, -0.03670336678624153, 0.018786832690238953, -0.016271045431494713, 0.0015455955872312188, -0.02287895977497101, -0.003919864073395729, 0.013413826003670692, 0.006799465976655483, -0.005125628784298897, -0.03806310519576073, -0.009914030320942402, 0.019929643720388412, -0.01597391627728939, -0.03921595960855484, 0.01797918602824211, -0.03757511451840401, -0.033648304641246796, -0.018641909584403038, 0.004708806052803993, 0.018534060567617416, 0.002305879956111312, 0.05184216797351837, 0.022654572501778603, 0.01962568610906601, 0.002412548055872321, -0.016045503318309784, 0.04193412885069847, 0.024484949186444283, -0.044798072427511215, -0.016152026131749153, 0.04343743249773979, 0.007785973139107227, 0.042069535702466965, 0.004373084753751755, -0.022511979565024376, -0.05041269585490227, -0.05011767894029617, -0.012169434688985348, -0.03610678389668465, -0.03506149724125862, -0.06163753569126129, -0.023480314761400223, -0.0038458623457700014, 0.03567172586917877, -0.003540861653164029, 0.017518483102321625, -0.008856534026563168, -0.05403071269392967, -0.0034520470071583986, -0.03720111772418022, -0.016007060185074806, -0.008873220533132553, -0.022026490420103073, -0.023521607741713524, 0.05590851232409477, -0.0004931605071760714, 0.052897050976753235, 0.027192985638976097, 0.006180732045322657, 0.02127205953001976, -0.035947732627391815, -0.03637014701962471, -0.06025996804237366, 0.0048209382221102715, -0.001171763171441853, 0.03144657984375954, 0.011349749751389027, -0.016657594591379166, 0.03373599797487259, -0.04576202854514122, 0.01021086797118187, -0.00757613405585289, -0.01253489963710308, -0.025599095970392227, -0.03161827102303505, 0.0562964491546154, -0.03356998413801193, 0.0165901780128479, -0.028999296948313713, 0.045910391956567764, 0.02929147705435753, 0.015289496630430222, -0.03967085853219032, -0.017411651089787483, -0.02390330284833908, -0.0383971706032753, 0.051908548921346664, -0.0041244071908295155, -0.02972569316625595, -0.05536685511469841, 0.004971896298229694, 0.061480648815631866, -0.019284488633275032, 0.021845996379852295, 0.03989787772297859, -0.014510140754282475, 0.013151167891919613, -0.02872444875538349, 0.03728681057691574, 0.027902944013476372, -0.017096269875764847, -0.008116280660033226, -0.05707154422998428, -0.03023579716682434, 0.013919169083237648, -0.045837294310331345, -0.05870185047388077, -0.04218591749668121, -0.005499355029314756, 0.06527318060398102, -0.009279721416532993, 0.04946454241871834, -0.016467086970806122, -0.027277477085590363, -0.03707776963710785, 0.06605549156665802, 0.016879627481102943, 0.01567702740430832, 0.010297874920070171, 0.014786206185817719, -0.008019065484404564, 0.02376498281955719, -0.017359241843223572, -0.04446140676736832, -0.0621924065053463, 0.07404530793428421, -0.02004597894847393, -0.05796246603131294, 0.0014579270500689745, 0.06070335954427719, -0.062255777418613434, -0.02822248823940754, -0.019852740690112114, -0.009004471823573112, 0.033728569746017456, -0.025793375447392464, 0.019177177920937538, -0.006851433776319027, 0.02970508486032486, 0.030179953202605247, 0.03527805581688881, -0.013606218621134758, 0.0431487150490284, 0.0010086470283567905, 0.04606986045837402, -0.015474792569875717, -0.005252708215266466, 0.025650765746831894, -0.033665720373392105, -0.003077165922150016, -0.019452586770057678, -0.017762647941708565, -0.015435555018484592, -0.0010351247619837523, 0.03982628509402275, 0.001056996756233275, -0.004833381623029709, 0.028836818411946297, -0.03229289874434471, 0.040309421718120575, -0.015145573765039444, 0.027250196784734726, 0.018166130408644676, -0.022787678986787796, -0.05943514406681061, -0.046602729707956314, 0.06222428008913994, 0.030677637085318565, 0.02076210081577301, -0.05279617756605148, -0.016923649236559868, 0.05188162252306938, 0.015383400954306126, -0.01435035653412342, -0.06151856854557991, -0.02672400139272213, -0.048457883298397064, -0.02827247604727745, -0.016949184238910675, -0.024282298982143402, 0.015110498294234276, 0.033530108630657196, 0.012918166816234589, -0.015784982591867447, 0.000676896539516747, -0.00679439352825284, -0.0016669087344780564, -0.011877770535647869, -0.018313201144337654, 0.029236217960715294, -0.031782366335392, -0.05707327276468277, -0.04415097087621689, -0.0068635703064501286, -0.014224007725715637, -0.008972174488008022, -0.040772974491119385, 0.007232875097543001, -0.036272693425416946, 0.01984979957342148, 0.009658361785113811, 0.041073255240917206, -0.006101801060140133, -0.02416880987584591, -0.019898654893040657, 0.012876676395535469, 0.00016556063201278448, 0.027064945548772812, 0.027218354865908623, -0.02053120546042919, 0.00884199794381857, 0.025847477838397026, -0.0183626227080822, -0.016478542238473892, 0.016405297443270683, 0.005429032724350691, -0.011533739045262337, -0.07160912454128265, -0.01083840150386095, -0.015387553721666336, -0.04261346161365509, 0.0324200764298439, -0.030311526730656624, 0.0020904135890305042, 0.011427994817495346, -0.011354411952197552, 0.012781173922121525, 0.05792901664972305, 0.009468398988246918, 0.057423919439315796, 0.012971822172403336, 0.028149956837296486, 0.022505884990096092, -0.009038998745381832, 0.028309572488069534, 0.05068499222397804, 0.02441098913550377, -0.004449956584721804, 0.0013302656589075923, 0.022844398394227028, -0.004337259102612734, 0.03343599662184715, -0.019148841500282288, -0.04196668788790703, -0.054949741810560226, -0.020762955769896507, -0.030894717201590538, 0.030482610687613487, 0.0028240755200386047, -0.014604011550545692, 0.053040388971567154, -0.04296177253127098, -0.02487114630639553, 0.008476530201733112, -0.05107930675148964, -0.028654105961322784, 0.024989651516079903, -0.020352430641651154, 0.020626064389944077, -0.009219222702085972, -0.03225496783852577, 0.0026903680991381407, -0.02461285889148712, -0.005108219105750322, -0.01833292469382286, 0.02293836697936058, -0.04259997978806496, 0.00858478806912899, 0.004541977774351835, -0.022543825209140778, 0.04408576339483261, 0.044215377420186996, -0.03558366373181343, -0.055398426949977875, 0.014820551499724388, -0.017116542905569077, 0.021352438256144524, 0.00898418202996254, -0.003302273340523243, 0.02605890855193138, -0.010787800885736942, 0.0009743990958668292, -0.008440643548965454, 0.01656857505440712, 0.0017043736297637224, 0.019138097763061523, -0.012315122410655022, -0.016400862485170364, -0.03827257826924324, 0.030305011197924614, 0.0038654799573123455, -0.0461978018283844, -0.012136342003941536, 0.0406438410282135, 0.02786164917051792, -0.024076636880636215, 0.018328983336687088, -0.015065009705722332, 0.030084604397416115, -0.03514893725514412, -0.01791638322174549, -0.022419355809688568, 0.017877116799354553, 0.013841292820870876, 0.020112590864300728, 0.01941867545247078, 0.018959594890475273, 0.04393429681658745, 0.02244577556848526, 0.013069085776805878, 0.03352518007159233, 0.05131212994456291, -0.03468658775091171, -0.012824981473386288, -0.019026294350624084, -0.007912250235676765, 0.006991938222199678, -0.007022670470178127, 0.011886459775269032, -0.026636524125933647, 0.0014336026506498456, -0.012980005703866482, -0.0426643043756485, 0.04091828316450119, -0.03653635457158089, 0.004646995570510626, -0.01764812134206295, 0.012330028228461742, 0.005317614413797855, 0.04854132980108261, 0.04241035133600235, 0.0031993642915040255, -2.4022041543503292e-05, 0.012300238013267517, -0.02156667970120907, 0.02942373976111412, 0.02931295521557331, -0.007913554087281227, 0.005917118862271309, 0.00796228926628828, 0.009082053788006306, -0.014223751612007618, -0.015265019610524178, 0.006661089137196541, -0.0328555628657341, 0.0371633805334568, -0.031423136591911316, -0.022577429190278053, 0.05065266042947769, -0.040365174412727356, -0.006416310556232929, -0.024516049772500992, 0.049850840121507645, -0.019526736810803413, -0.005364623386412859, 0.021171536296606064, 0.01772318407893181, -0.03629846125841141, 0.06399472802877426, -0.013322308659553528, 0.0584615059196949, 0.0025386761408299208, 0.057182833552360535, -0.018492452800273895, 0.04985341429710388, 0.017360148951411247, -0.030813002958893776, -0.02299424260854721, -0.0005577250267378986, 0.0032063110265880823, -0.052540719509124756, 0.004513117019087076, -0.023615268990397453, -0.010156054049730301, -0.05734248831868172, -0.0034615786280483007, -0.00840687658637762, 0.042960021644830704, -0.013433185406029224, -0.03933856636285782, -0.00243240874260664, 0.022423943504691124, -0.03300131484866142, -0.007336396258324385, 0.007686299737542868, 0.056139953434467316, -0.006219574250280857, -0.012558910995721817, -0.0162200890481472, -0.03473757579922676, -0.024645088240504265, -0.02992498315870762, 0.08166597038507462, -0.007272781804203987, -0.03045867197215557, -0.039733465760946274, -0.05261974036693573, 0.0060821957886219025, 0.007268948946148157, -0.04023372754454613, -0.030833439901471138, 0.06685485690832138, 0.04595223069190979, -0.0033289585262537003, -0.006033727899193764, -0.03473981097340584, 0.022295743227005005, 0.022339927032589912, 0.06572134792804718, -0.011759467422962189, 0.056051820516586304, 0.04736318811774254, -0.052341241389513016, -0.0013137281639501452, -0.025487635284662247, 0.03562435507774353, -0.010556125082075596, 0.013089721091091633, -0.018666012212634087, 0.03802625834941864, -0.002297690836712718, -0.03506523370742798, 0.032235369086265564, 0.021482843905687332, 0.008531543426215649, -0.042011555284261703, -0.09686960279941559, 0.00405152328312397, -0.07200459390878677, -0.00779748847708106, -0.06629585474729538, 0.024881139397621155, 0.00783358421176672, 0.009105878882110119, -0.007014678791165352, -0.05316596478223801, 0.15715666115283966, 0.026312783360481262, 0.04170495271682739, -0.0024423825088888407, 0.03255728632211685, 0.04389331489801407, -0.0033591107930988073, -0.0044932602904737, 0.00027515762485563755, -0.015456654131412506, 0.02745087631046772, 0.02001403272151947, 0.039437443017959595, -0.015141886658966541, 0.016828017309308052, 0.0350237712264061, -0.026708193123340607, 0.024615582078695297, 0.02645600028336048, -0.02915438823401928, -0.07947345077991486, 0.019846046343445778, 0.016488824039697647, 0.025612076744437218, 0.0023415074683725834, -0.002516100648790598, -0.00579975126311183, -0.05305306240916252, -0.008046052418649197, -0.015546994283795357, 0.03372327983379364, -0.041342880576848984, 0.02370927855372429, -0.005892025772482157, -0.044583678245544434, 0.05119594559073448, 0.023535238578915596, -0.014886204153299332, 0.0010607045842334628, 0.014597799628973007, -0.04223111271858215, -0.010579951107501984, 0.026145394891500473, -0.04237351194024086, -0.002020868007093668, 0.02380148135125637, -0.04308566078543663, 0.006028672680258751, 0.000558320025447756, -0.035066910088062286, 0.06088153272867203, -0.05992026627063751, 0.019113434478640556, -0.01665104739367962, -0.04205554723739624, 0.02525733783841133, 0.017619462683796883, -0.019967960193753242, -0.018884116783738136, 0.02735177055001259, 0.022821063175797462, 0.00044950968003831804, -0.017049498856067657, -0.02620229497551918, -0.011528629809617996, -0.0010521435178816319, 0.014900805428624153, -0.006195124238729477, 0.01612023264169693, -0.03386173024773598, -0.010201094672083855, -0.017007552087306976, 0.004599532578140497, -0.003167906543239951, 0.04364302009344101, 0.0144586730748415, -0.014103000983595848, 0.052093490958213806, 0.00992075726389885, -0.04009290412068367, -0.00022297556279227138, -0.04304269701242447, -0.006559044122695923, 0.004573266953229904, 0.03959871083498001, 0.03816087916493416, -0.04470319300889969, -0.041086699813604355, -0.01534073706716299, 0.06998822838068008, 0.037421099841594696, 0.06769272685050964, -0.010106327943503857, 0.00033660681219771504, -0.017923248931765556]

In [ ]:
results = collection.aggregate([
  {
    "$vectorSearch": {
      "index": "vector_search_index",
      "path": "description_embedding",
      "queryVector": query_vector,
      "numCandidates": 100,
      "limit": 10
    }
  }
])

In [ ]:
for i in results:
    del i['description_embedding']
    del i['name_embedding']
    print(i)